In [79]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re

# code가지고 오기

In [80]:
url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
payload = {
    'locale' : 'ko_KR',
    'share' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT04601'
}

headers = {
    'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'
}
code = rq.post(url,payload,headers = headers).text


# 현재 모든 etf데이터 가지고 오기

In [81]:
url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
payload = {
    'code' : code
}

etfs = rq.post(url , payload, headers = headers)

# 데이터를 csv로 처리하기( 현재 데이터의 content는 html로 파싱되어 있음 )

In [82]:
from io import BytesIO
etfs = pd.read_csv(BytesIO(etfs.content),encoding = 'EUC_KR')

In [83]:
etfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856 entries, 0 to 855
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   표준코드    856 non-null    object 
 1   단축코드    856 non-null    int64  
 2   한글종목명   856 non-null    object 
 3   한글종목약명  856 non-null    object 
 4   영문종목명   856 non-null    object 
 5   상장일     856 non-null    object 
 6   기초지수명   856 non-null    object 
 7   지수산출기관  856 non-null    object 
 8   추적배수    856 non-null    object 
 9   복제방법    856 non-null    object 
 10  기초시장분류  856 non-null    object 
 11  기초자산분류  856 non-null    object 
 12  상장좌수    856 non-null    int64  
 13  운용사     856 non-null    object 
 14  CU수량    856 non-null    int64  
 15  총보수     856 non-null    float64
 16  과세유형    856 non-null    object 
dtypes: float64(1), int64(3), object(13)
memory usage: 113.8+ KB


In [84]:
etfs.isnull().sum()

표준코드      0
단축코드      0
한글종목명     0
한글종목약명    0
영문종목명     0
상장일       0
기초지수명     0
지수산출기관    0
추적배수      0
복제방법      0
기초시장분류    0
기초자산분류    0
상장좌수      0
운용사       0
CU수량      0
총보수       0
과세유형      0
dtype: int64

In [85]:
etfs.head()

,표준코드,단축코드,한글종목명,한글종목약명,영문종목명,상장일,기초지수명,지수산출기관,추적배수,복제방법,기초시장분류,기초자산분류,상장좌수,운용사,CU수량,총보수,과세유형
0,KR7470320003,470320,BNK BNK 24-11 회사채(AA-이상)액티브증권상장지수투자신탁(채권),BNK 24-11 회사채(AA-이상)액티브,BNK BNK 24-11 Corporate Bond(AA- or higher) Ac...,2023/11/14,KAP 24-11 회사채 총수익 지수(AA-이상),한국자산평가(KAP),일반,실물(액티브),국내,채권,2134000,비엔케이자산운용,2000,0.100,배당소득세(보유기간과세)
1,KR7466810009,466810,BNK BNK 2차전지양극재증권상장지수투자신탁(주식),BNK 2차전지양극재,BNK BNK Secondary Battery Cathode Materials ETF,2023/10/19,iSelect 2차전지양극재 지수,NH투자증권,일반,실물(패시브),국내,주식,750000,비엔케이자산운용,50000,0.395,비과세
2,KR7457930006,457930,BNK BNK 미래전략기술액티브증권상장지수투자신탁(주식),BNK 미래전략기술액티브,BNK BNK Future Strategic Technology Active ETF,2023/06/20,iSelect 미래전략기술 지수 (PR),NH투자증권,일반,실물(액티브),국내,주식,750000,비엔케이자산운용,50000,0.495,배당소득세(보유기간과세)
3,KR7445690001,445690,BNK BNK 주주가치액티브증권상장지수투자신탁(주식),BNK 주주가치액티브,BNK BNK Shareholder Yield Active ETF,2022/10/27,FnGuide 주주가치 지수,FnGuide,일반,실물(액티브),국내,주식,1100000,비엔케이자산운용,50000,0.495,배당소득세(보유기간과세)
4,KR7465780005,465780,DB 마이티 26-09 특수채(AAA)액티브증권상장지수투자신탁[채권],마이티 26-09 특수채(AAA)액티브,DB Mighty 26-09 Special Bond Active ETF,2023/09/21,KAP 26-09 특수채 지수(총수익),한국자산평가(KAP),일반,실물(액티브),국내,채권,730000,디비자산운용,1000,0.060,배당소득세(보유기간과세)


In [88]:
etf_info = etfs[['표준코드','단축코드' , '한글종목명','상장일']]
etf_info = etf_info.rename(columns = {'한글종목명' : '종목명'})

etf_info['상장일'] = pd.to_datetime(etf_info['상장일'], format='%Y/%m/%d')


etf_info

,표준코드,단축코드,종목명,상장일
0,KR7470320003,470320,BNK BNK 24-11 회사채(AA-이상)액티브증권상장지수투자신탁(채권),2023-11-14
1,KR7466810009,466810,BNK BNK 2차전지양극재증권상장지수투자신탁(주식),2023-10-19
2,KR7457930006,457930,BNK BNK 미래전략기술액티브증권상장지수투자신탁(주식),2023-06-20
3,KR7445690001,445690,BNK BNK 주주가치액티브증권상장지수투자신탁(주식),2022-10-27
4,KR7465780005,465780,DB 마이티 26-09 특수채(AAA)액티브증권상장지수투자신탁[채권],2023-09-21
...,...,...,...,...
851,KR7476000005,476000,현대 UNICORN 포스트IPO액티브증권상장지수투자신탁[주식],2024-02-27
852,KR7215620006,215620,흥국 HK S&P로우볼증권상장지수투자신탁(주식),2015-03-25
853,KR7391670007,391670,흥국 HK 베스트일레븐액티브증권상장지수투자신탁(주식),2021-07-06
854,KR7472920008,472920,흥국 HK 종합채권(AA-이상)액티브증권상장지수투자신탁[채권],2023-11-28


## DB연결

In [89]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time

In [96]:
engine = create_engine('mysql+pymysql://user:password@ip:3306/db')
con = pymysql.connect(user = user,
                      password = password,
                      host = host,
                      db = db,
                      charset = 'utf8')
mycursor = con.cursor()



In [97]:
query = """
insert into etf_info (표준코드 , 단축코드 , 종목명, 상장일)
values (%s,%s,%s,%s)
"""
values_list = []
for index, row in tqdm(etf_info.iterrows()):
    values_list.append((row['표준코드'] , row['단축코드'] , row['종목명'] , row['상장일']))


mycursor.executemany(query , values_list)
con.commit()

engine.dispose()
con.close()

856it [00:00, 19028.64it/s]
